In [ ]:
import constants
import pandas  as pd
from tqdm import tqdm
import hashlib
import collections
import itertools
import functools
import seaborn as sns
import pickle
import safer
import matplotlib.pyplot as plt
import random

In [ ]:
sns.set_style(style="whitegrid")

In [ ]:
constants.set_crawl_source(constants.CRAWL_SOURCE_CHICAGO)

In [ ]:
tqdm.pandas()

In [ ]:
df = pd.read_pickle(constants.LONG_DATA_FILE)

In [ ]:
stats_df = pd.read_pickle(constants.RECALSSIFICATION_DATA_FILE)
#stats_df = stats_df.reset_index(level=[1])

In [ ]:
#df_hash_index = df.set_index("content_hash",append=True,drop=False).reorder_levels([1,0]).sort_index(level=[0])
#df_hash_index = df_hash_index[~df_hash_index.index.duplicated(keep='first')]

In [ ]:
def fn(series):
    content_hash = series.name
    row = df_hash_index.xs(content_hash,level=0).iloc[0]
    date = row.date
    business_id = row.business_id
    return pd.Series([date,business_id])
    
#s = stats_df.progress_apply(fn,axis=1)

In [ ]:
#s.rename({0:"date", 1:"business_id"},axis=1)
#stats_df["date"] = s[0]
#stats_df["business_id"] = s[1]

In [ ]:
stats_df.reclassification_swaps.value_counts()

In [ ]:
stats_df["reclass_swaps_joined"] = stats_df.reclassification_swaps.replace({0:"0",1:"1",2:"2",3:"3+",4:"3+",5:"3+"})
swap = {k: f"{k} ({v:,})" for k,v in stats_df["reclass_swaps_joined"].value_counts().to_dict().items()}
stats_df["Number of reclassifications"] = stats_df["reclass_swaps_joined"].replace(swap)
sns.ecdfplot(data=stats_df, x="date", hue="Number of reclassifications").set(xlabel="Date")
plt.savefig(f"../../graphs/reclassification_by_date_grouped_{constants.CRAWL_SOURCE}.pdf",bbox_inches='tight')

In [ ]:
sns.ecdfplot(data=stats_df, x="date", hue="reclassification_swaps")
plt.savefig(f"../../graphs/reclassification_by_date_{constants.CRAWL_SOURCE}.pdf",bbox_inches='tight')

# Worst case reclassification

In [ ]:
#df.groupby("business_id","crawl_id")

In [ ]:
differences = []
for bid, bid_df in df.groupby("business_id"):
    prev_rating = None
    prev_crawl = None
    prev_review_count = None
    
    for crawl, reviews in bid_df.groupby("crawl_id"):
        if len(reviews[reviews.flagged==False]) == 0:
            continue
        rating = reviews[reviews.flagged==False].rating.mean()
        review_count = len(reviews)
        if prev_rating is not None and prev_review_count == review_count:
            differences.append((bid,prev_crawl,crawl,review_count,rating-prev_rating))
        prev_rating = rating
        prev_crawl = crawl
        prev_review_count = review_count
        
diff_df = pd.DataFrame(differences,columns=["business_id","previous_crawl_id","crawl_id","review_count","diff"])

In [ ]:
diff_df["abs_diff"] = diff_df["diff"].abs()

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)

In [ ]:
diff_df = diff_df.sort_values("diff",ascending=False)
diff_df

In [ ]:
rowid = 15919
print(diff_df.loc[rowid].business_id)
business_reviews = df[(df.business_id==diff_df.loc[rowid].business_id)][["content","rating","date","user_name","flagged","crawl_id"]]
display(business_reviews[business_reviews.crawl_id == diff_df.loc[rowid].previous_crawl_id])
display(business_reviews[business_reviews.crawl_id == diff_df.loc[rowid].crawl_id])

In [ ]:
sns.ecdfplot(x="abs_diff",data=diff_df)
plt.yscale('log')